In [1]:
%%capture
!uv pip install torch tokenizers numpy datasets transformers[torch] accelerate

In [2]:
import torch

import numpy as np
import torch.nn as nn

from torch.nn import functional as F

from datasets import load_from_disk

from transformers import PreTrainedTokenizerFast

In [3]:
from kn1ght.constants import SPECIAL_TOKENS

DATASET_PATH = "../.data/datasets/full/2025-01-20"
TOKENIZER_PATH = "../src/tokenizer/kn1ght-tokenizer.json"

In [4]:
HYPERPARAMETERS = {
    "batch_size": 8,
    "block_size": 32,
    "max_iters": 5000,
    "max_length": 32,
    "eval_interval": 100,
    "learning_rate": 1e-3,
    "device": (
        "cuda"
        if torch.cuda.is_available()
        else "mps" if torch.backends.mps.is_available() else "cpu"
    ),
    "eval_iters": 200,
    "n_embed": 64,
    "n_head": 4,
    "n_layer": 4,
    "dropout": 0.0,
}

In [5]:
raw_dataset = load_from_disk(DATASET_PATH)

In [6]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_PATH)

In [7]:
tokenizer.pad_token = SPECIAL_TOKENS["PAD"]

In [8]:
def sample_data(dataset, samples=5000) -> dict:
    return dataset.shuffle(seed=1997).select(range(samples))

In [9]:
def tokenize(examples):
    return tokenizer(
        examples["PGN"],
        return_special_tokens_mask=True,
        max_length=HYPERPARAMETERS["max_length"],
        truncation=True,
        padding_side="right",
        padding="max_length",
    )

In [10]:
torch.manual_seed(1997)

torch.set_default_device(HYPERPARAMETERS["device"])

In [11]:
sample = sample_data(raw_dataset["train"])

data = tokenize(sample)["input_ids"]

tensor_data = torch.tensor(data, dtype=torch.long)

n = int(0.9 * len(tensor_data))

train_data = tensor_data[:n]
val_data = tensor_data[n:]

In [12]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data

    ix = torch.randint(
        len(data) - HYPERPARAMETERS["block_size"],
        (HYPERPARAMETERS["batch_size"],),
    )

    x = torch.stack([data[i : i + HYPERPARAMETERS["block_size"]] for i in ix])
    y = torch.stack([data[i + 1 : i + HYPERPARAMETERS["block_size"] + 1] for i in ix])

    x, y = x.to(HYPERPARAMETERS["device"]), y.to(HYPERPARAMETERS["device"])

    return x, y

In [13]:
class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(
            HYPERPARAMETERS["n_embed"],
            head_size,
            bias=False,
        )
        self.query = nn.Linear(
            HYPERPARAMETERS["n_embed"],
            head_size,
            bias=False,
        )
        self.value = nn.Linear(
            HYPERPARAMETERS["n_embed"],
            head_size,
            bias=False,
        )
        # self.register_buffer(
        #     "tril",
        #     torch.tril(
        #         torch.ones(HYPERPARAMETERS["block_size"], HYPERPARAMETERS["block_size"])
        #     ),
        # )

        self.dropout = nn.Dropout(HYPERPARAMETERS["dropout"])

    def forward(self, x):
        B, T, C, Z = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [14]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(
            HYPERPARAMETERS["n_embed"],
            HYPERPARAMETERS["n_embed"],
        )
        self.dropout = nn.Dropout(HYPERPARAMETERS["dropout"])

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [15]:
class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(HYPERPARAMETERS["dropout"]),
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [17]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(
            tokenizer.vocab_size, HYPERPARAMETERS["n_embed"]
        )
        self.position_embedding_table = nn.Embedding(
            HYPERPARAMETERS["block_size"], HYPERPARAMETERS["n_embed"]
        )
        self.blocks = nn.Sequential(
            *[
                Block(HYPERPARAMETERS["n_embed"], n_head=HYPERPARAMETERS["n_head"])
                for _ in range(HYPERPARAMETERS["n_layer"])
            ]
        )
        self.ln_f = nn.LayerNorm(HYPERPARAMETERS["n_embed"])  # final layer norm
        self.lm_head = nn.Linear(
            HYPERPARAMETERS["n_embed"],
            tokenizer.vocab_size,
        )

    def forward(self, idx, targets=None):
        B, T, C = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C, Z = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -HYPERPARAMETERS["block_size"] :]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [18]:
model = BigramLanguageModel()
# m = model.to(HYPERPARAMETERS["device"])

In [19]:
# print the number of parameters in the model
parameters = sum(p.numel() for p in model.parameters())

print(
    f"{round(parameters / 1e6)}M parameters"
    if parameters > 1e6
    else f"{round(parameters / 1e4)}K parameters"
)

73K parameters


In [20]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(HYPERPARAMETERS["eval_iters"])
        for k in range(HYPERPARAMETERS["eval_iters"]):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
for iter in range(HYPERPARAMETERS["max_iters"]):

    # every once in a while evaluate the loss on train and val sets
    if (
        iter % HYPERPARAMETERS["eval_interval"] == 0
        or iter == HYPERPARAMETERS["max_iters"] - 1
    ):
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

RuntimeError: shape '[256, 32]' is invalid for input of size 33554432

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long)
print(tokenizer.decode(model.generate(context, max_new_tokens=1)[0].tolist()))